In [19]:
import pandas as pd
import ast

In [3]:
expected_DynamicO2O = 400

##### Dirigo

In [4]:
dirigo_df = pd.read_csv('../Dirigo/OCEL_O2O.csv')

In [8]:
grouped_dirigo = dirigo_df.groupby(['Source_object_id', 'Target_object_id'])['Timestamp'].nunique().reset_index()
grouped_dirigo

,Source_object_id,Target_object_id,Timestamp
0,Pcp1,Cr4,1
1,Pcp10,Cr5,1
2,Pcp100,Cr20,1
3,Pcp11,Cr19,1
4,Pcp12,Cr9,1
...,...,...,...
295,tr8,Pcp77,2
296,tr8,Pcp80,2
297,tr9,Pcp13,2
298,tr9,Pcp6,2


In [9]:
dynamic_pairs = grouped_dirigo[grouped_dirigo['Timestamp'] > 1][['Source_object_id', 'Target_object_id']]
dynamic_pairs


,Source_object_id,Target_object_id
100,tr1,Pcp24
101,tr1,Pcp46
102,tr1,Pcp47
103,tr1,Pcp96
104,tr10,Pcp28
...,...,...
295,tr8,Pcp77
296,tr8,Pcp80
297,tr9,Pcp13
298,tr9,Pcp6


In [13]:
dynamic_pairs_dirigo = pd.merge(dirigo_df[['Source_object_id', 'Target_object_id','Timestamp']], dynamic_pairs, on=['Source_object_id', 'Target_object_id'], how='inner')
dynamic_pairs_dirigo

,Source_object_id,Target_object_id,Timestamp
0,tr26,Pcp44,2024-05-01 08:05:48
1,tr49,Pcp44,2024-05-01 08:08:49
2,tr41,Pcp87,2024-05-01 08:13:40
3,tr22,Pcp87,2024-05-01 08:15:10
4,tr11,Pcp48,2024-05-01 08:32:57
...,...,...,...
395,tr29,Pcp51,2024-05-02 15:18:40
396,tr29,Pcp70,2024-05-03 15:08:59
397,tr49,Pcp70,2024-05-03 15:09:23
398,tr49,Pcp70,2024-05-03 15:38:46


In [26]:
completeness_ratio_dirigo = len(dynamic_pairs_dirigo) / expected_DynamicO2O
print(f"DO2OCR (Dirigo): {completeness_ratio_dirigo:.2%}")

DO2OCR (Dirigo): 100.00%


##### ACEL

In [16]:
acel_events = pd.read_csv('../ACEL/ACEL_Events.csv')

In [20]:
relation_changes = []
for _, row in acel_events.iterrows():
        changes_raw = row['RelationChanges']
        if pd.notna(changes_raw):
            try:
                changes = ast.literal_eval(changes_raw)
                for change in changes:
                    relation_changes.append({
                        'RelationId': change.get('RelationId'),
                        'ChangeStatus': change.get('ChangeStatus')
                    })
            except Exception:
                continue
acel_o2o = pd.DataFrame(relation_changes)
acel_o2o

,RelationId,ChangeStatus
0,Cr2_to_Pcp44,addedTarget
1,Cr9_to_Pcp87,addedTarget
2,Cr20_to_Pcp48,addedTarget
3,Cr13_to_Pcp55,addedTarget
4,Cr7_to_Pcp69,addedTarget
...,...,...
495,Pcp51_to_tr29,deletedTarget
496,Pcp70_to_tr29,addedTarget
497,Pcp70_to_tr49,addedTarget
498,Pcp70_to_tr49,deletedTarget


In [25]:
relation_counts = acel_o2o.groupby('RelationId')['ChangeStatus'].nunique().reset_index()

dynamic_relation_ids = relation_counts[relation_counts['ChangeStatus'] > 1]['RelationId']


acel_dynamic_relations = acel_o2o[acel_o2o['RelationId'].isin(dynamic_relation_ids)]


completeness_ratio_acel = len(acel_dynamic_relations) / expected_DynamicO2O

print(f"DO2OCR (ACEL): {completeness_ratio_acel:.2%}")

DO2OCR (ACEL): 100.00%


##### DOCEL

In [32]:
docel_df = pd.read_csv('../DOCEL/DOCEL_TruckPickupplanID.csv')
docel_df

,TPcpID,Pickup Plan ID,objectID,eventID
0,Pickup Plan ID_0,Pcp44,tr26,assign_tr26_Pcp44
1,Pickup Plan ID_1,Pcp44,tr49,assign_tr49_Pcp44
2,Pickup Plan ID_2,Pcp87,tr41,assign_tr41_Pcp87
3,Pickup Plan ID_3,Pcp87,tr22,assign_tr22_Pcp87
4,Pickup Plan ID_4,Pcp48,tr11,assign_tr11_Pcp48
...,...,...,...,...
195,Pickup Plan ID_195,Pcp77,tr17,assign_tr17_Pcp77
196,Pickup Plan ID_196,Pcp51,tr23,assign_tr23_Pcp51
197,Pickup Plan ID_197,Pcp51,tr29,assign_tr29_Pcp51
198,Pickup Plan ID_198,Pcp70,tr29,assign_tr29_Pcp70


In [35]:
unique_pairs = docel_df[['Pickup Plan ID','objectID']].drop_duplicates()
len(unique_pairs)

200

In [37]:
completeness_ratio_docel = len(unique_pairs) / expected_DynamicO2O
print(f"DO2OCR (DOCEL): {completeness_ratio_docel:.2%}")

DO2OCR (DOCEL): 50.00%
